In [7]:
import pandas as pd
import requests
from census import Census
from config import api_key
import pymongo
from sqlalchemy import create_engine
c = Census(api_key, year=2018)

## ETL 1 - Data extracted from an API and stored in a postgres Database

## Step1. Extract from Census JSON API     

In [8]:
#call census api by zipcode 
census_zip = c.acs5.get(("B19013_001E", 
                                     "B01003_001E", 
                                     "B01002_001E",
                                     "B19301_001E",
                                     "B17001_002E",
                                     "B23025_004E",
                                     "B23025_002E",
                                     "B23025_005E"),
                                    {'for': 'zip code tabulation area:*'})
census_zip

[{'B19013_001E': 13092.0,
  'B01003_001E': 17242.0,
  'B01002_001E': 40.5,
  'B19301_001E': 6999.0,
  'B17001_002E': 10772.0,
  'B23025_004E': 3495.0,
  'B23025_002E': 5811.0,
  'B23025_005E': 2316.0,
  'zip code tabulation area': '00601'},
 {'B19013_001E': 16358.0,
  'B01003_001E': 38442.0,
  'B01002_001E': 42.3,
  'B19301_001E': 9277.0,
  'B17001_002E': 19611.0,
  'B23025_004E': 11536.0,
  'B23025_002E': 13493.0,
  'B23025_005E': 1927.0,
  'zip code tabulation area': '00602'},
 {'B19013_001E': 16603.0,
  'B01003_001E': 48814.0,
  'B01002_001E': 41.1,
  'B19301_001E': 11307.0,
  'B17001_002E': 24337.0,
  'B23025_004E': 12400.0,
  'B23025_002E': 15595.0,
  'B23025_005E': 3124.0,
  'zip code tabulation area': '00603'},
 {'B19013_001E': 12832.0,
  'B01003_001E': 6437.0,
  'B01002_001E': 43.3,
  'B19301_001E': 5943.0,
  'B17001_002E': 4163.0,
  'B23025_004E': 1355.0,
  'B23025_002E': 1585.0,
  'B23025_005E': 230.0,
  'zip code tabulation area': '00606'},
 {'B19013_001E': 19309.0,
  'B0100

## Step 2. Transform the data by cleaning and structure to the desired format

In [3]:
#create dataframe 
census_zip = pd.DataFrame(census_zip)
#rename 
census_zip = census_zip.rename(columns={
                                      "B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_004E": "employment_employed",
                                      "B23025_002E": "Labor_force",
                                      "B23025_005E": "employment_unemployed",
                                    "zip code tabulation area": "zipcode"})
census_zip = census_zip.dropna()
census_zip

,Household Income,Population,Median Age,Per Capita Income,Poverty Count,employment_employed,Labor_force,employment_unemployed,zipcode
0,13092.0,17242.0,40.5,6999.0,10772.0,3495.0,5811.0,2316.0,00601
1,16358.0,38442.0,42.3,9277.0,19611.0,11536.0,13493.0,1927.0,00602
2,16603.0,48814.0,41.1,11307.0,24337.0,12400.0,15595.0,3124.0,00603
3,12832.0,6437.0,43.3,5943.0,4163.0,1355.0,1585.0,230.0,00606
4,19309.0,27073.0,42.1,10220.0,11724.0,8464.0,9754.0,1290.0,00610
...,...,...,...,...,...,...,...,...,...
33079,34028.0,330.0,39.5,18213.0,129.0,106.0,117.0,11.0,99922
33081,57375.0,927.0,43.6,25840.0,172.0,390.0,479.0,89.0,99925
33082,53409.0,1635.0,34.5,22453.0,235.0,540.0,678.0,138.0,99926
33083,-666666666.0,38.0,55.5,13658.0,28.0,28.0,28.0,0.0,99927


## ELT 1
## Extraction 2  
## Step 1 - Covid-19 data from Santa Clara County JSON

In [4]:
url = "https://data.sccgov.org/resource/j2gj-bg6c.json"
response = requests.get(url).json() 
#create dataframe 
santa_clara_covid_zipcode = pd.DataFrame(response)
santa_clara_covid_zipcode.head()

,zipcode,cases,population,rate
0,94022,65,19378,335
1,94024,67,23961,280
2,94040,195,35845,544
3,94041,92,14394,639
4,94043,146,31488,464



## Step 2. Transform - clean and structure the data in the desired form from the covid-19 dataset.

In [6]:
#rename columns
santa_clara_covid_zipcode = santa_clara_covid_zipcode.rename(\
                    columns={"cases": "infected count" #rename column
                            })
#Insert infect count and infect rate columns 
santa_clara_covid_zipcode['infected count'] = \
    santa_clara_covid_zipcode['infected count'].fillna(0)
santa_clara_covid_zipcode['rate'] = \
    santa_clara_covid_zipcode['rate'].fillna(0)
santa_clara_covid_zipcode["infected percent"] =\
    100 * santa_clara_covid_zipcode["infected count"].astype(int)/santa_clara_covid_zipcode["population"].astype(int)
santa_clara_covid_zipcode.head()

,zipcode,infected count,population,rate,infected rate,infected percent
0,94022,65,19378,335,0.335432,0.335432
1,94024,67,23961,280,0.279621,0.279621
2,94040,195,35845,544,0.544009,0.544009
3,94041,92,14394,639,0.639155,0.639155
4,94043,146,31488,464,0.463669,0.463669


## Transform data by merging Census JSON and Santa Clara JSON dataframe

In [56]:
santa_clara_covid_demographics = pd.merge(santa_clara_covid_zipcode, census_zip, how="left", on = ["zipcode","zipcode"])
santa_clara_covid_demographics()

,zipcode,infected count,population,rate,infected rate,Household Income,Population,Median Age,Per Capita Income,Poverty Count,employment_employed,Labor_force,employment_unemployed
0,94022,64,19378,330,0.330271,216042.0,19378.0,48.1,122105.0,833.0,8111.0,8370.0,259.0
1,94024,57,23961,238,0.237887,237454.0,23961.0,46.2,106527.0,679.0,10643.0,11085.0,425.0
2,94040,185,35845,516,0.516111,129188.0,35845.0,35.1,73521.0,2681.0,19591.0,20335.0,744.0
3,94041,85,14394,591,0.590524,129444.0,14394.0,34.3,75692.0,946.0,9033.0,9410.0,377.0
4,94043,139,31488,441,0.441438,125916.0,31488.0,34.7,71492.0,2046.0,18620.0,19404.0,623.0


## Step 3. Load data into Postgres database

### Connect to local database

In [17]:
engine = create_engine(f'postgresql://postgres:@localhost/covid19_santa_claraDB')

In [18]:
engine.table_names()

[]

In [30]:
santa_clara_covid_demographics.to_sql(name='covid19', con=engine, if_exists='replace', index=False)

### Confirm data has been added by querying the covid19 table

In [31]:
pd.read_sql_query('select * from covid19', con=engine).head()

,zipcode,infect count,population,rate,infect rate,Household Income,Population,Median Age,Per Capita Income,Poverty Count,employment_employed,Labor_force,employment_unemployed
0,94022,64,19378,330,0.330271,216042.0,19378.0,48.1,122105.0,833.0,8111.0,8370.0,259.0
1,94024,57,23961,238,0.237887,237454.0,23961.0,46.2,106527.0,679.0,10643.0,11085.0,425.0
2,94040,185,35845,516,0.516111,129188.0,35845.0,35.1,73521.0,2681.0,19591.0,20335.0,744.0
3,94041,85,14394,591,0.590524,129444.0,14394.0,34.3,75692.0,946.0,9033.0,9410.0,377.0
4,94043,139,31488,441,0.441438,125916.0,31488.0,34.7,71492.0,2046.0,18620.0,19404.0,623.0


## ELT 2. 
## Step 1. Extract demographics data. Scrape the articles on the website http://www.mapszipcode.com/

In [32]:
from bs4 import BeautifulSoup as bs
import requests

In [33]:
zipcodes = santa_clara_covid_zipcode['zipcode'].to_list()
zipcode_list = []
for zipcode in zipcodes:
    base_url = 'http://www.mapszipcode.com/california/ladera%20ranch/'
    zipcodeDict={}
    response = requests.get(base_url + zipcode)
    soup = bs(response.text, 'html.parser')
    results = soup.find_all('div', class_="dat")
    zipcodeDict['zipcode'] = zipcode
    zipcodeDict["population"] = results[0].text.split(' ', 1 )[0]
    zipcodeDict["density"] = results[1].text.split(' ', 1 )[0].replace('\t', '').replace('/', '')
    zipcode_list.append(zipcodeDict)
zipcode_list    

[{'zipcode': '94022', 'population': '19,310', 'density': '1,104.17'},
 {'zipcode': '94024', 'population': '22,536', 'density': '3,086.45'},
 {'zipcode': '94040', 'population': '33,918', 'density': '9,194.61'},
 {'zipcode': '94041', 'population': '12,929', 'density': '8,451.43'},
 {'zipcode': '94043', 'population': '27,279', 'density': '2,555.84'},
 {'zipcode': '94085', 'population': '0', 'density': '0.00'},
 {'zipcode': '94086', 'population': '45,531', 'density': '10,150.25'},
 {'zipcode': '94087', 'population': '53,446', 'density': '8,119.28'},
 {'zipcode': '94089', 'population': '20,282', 'density': '3,384.56'},
 {'zipcode': '94301', 'population': '16,819', 'density': '7,087.06'},
 {'zipcode': '94304', 'population': '3,791', 'density': '583.39'},
 {'zipcode': '94305', 'population': '13,541', 'density': '2,640.81'},
 {'zipcode': '94306', 'population': '26,448', 'density': '6,317.15'},
 {'zipcode': '95002', 'population': '1,906', 'density': '159.21'},
 {'zipcode': '95008', 'population'

## Step 2. Second extraction to obtain income per family in Santa Clara County

In [34]:
for zipcodeDict in zipcode_list:
    response = requests.get('http://www.mapszipcode.com/california/ladera%20ranch/' + zipcodeDict['zipcode'] +  '/demographics/')
    #BeautifulSoup object
    soup = bs(response.text, 'html.parser')

    #Retrieve the latest subject and content from the Mars website
    results = soup.find_all('div', class_="col span_12_of_12")
    income={}
    for result in results:  
                    
        if result.find("h2"):
            if result.find("h2").text.find('Family and household') == 0:
                sub_results = result.find("tbody").find_all("tr")
                for sub_result in sub_results:
                    dictResult = sub_result.find_all('td')
                    income[dictResult[0].contents[0].replace('$', 'USD')] = dictResult[1].contents[0]              
    zipcodeDict['income'] = income


In [35]:
zipcode_list

[{'zipcode': '94022',
  'population': '19,310',
  'density': '1,104.17',
  'income': {'USD30,000 or less': '4.51%',
   'USD30,000 and USD50,000': '5.67%',
   'USD50,000 and USD100,000': '11.6%',
   'USD100,000 and USD200,000': '28.45%',
   'USD200,000 or more': '49.77%'}},
 {'zipcode': '94024',
  'population': '22,536',
  'density': '3,086.45',
  'income': {'USD30,000 or less': '2.58%',
   'USD30,000 and USD50,000': '4.99%',
   'USD50,000 and USD100,000': '13.28%',
   'USD100,000 and USD200,000': '31.08%',
   'USD200,000 or more': '48.07%'}},
 {'zipcode': '94040',
  'population': '33,918',
  'density': '9,194.61',
  'income': {'USD30,000 or less': '11.93%',
   'USD30,000 and USD50,000': '8.22%',
   'USD50,000 and USD100,000': '21.89%',
   'USD100,000 and USD200,000': '30.68%',
   'USD200,000 or more': '27.29%'}},
 {'zipcode': '94041',
  'population': '12,929',
  'density': '8,451.43',
  'income': {'USD30,000 or less': '16.92%',
   'USD30,000 and USD50,000': '11.57%',
   'USD50,000 and 

## Step 3. Load data to Mongo database

In [36]:
client = pymongo.MongoClient('mongodb://localhost:27017')
#define the database name
mydb = client["demographics_zip_DB"]
#define the collection name
mycol = mydb["demographics_zip"] 
mycol.insert_many(zipcode_list)

In [37]:
mydoc = mycol.find()
for x in mydoc:
    print("Zip Code: {0} \nPopulation: {1} \nDensity {2} ".format(x["zipcode"],x['population'],x['density']))
    print("The income(USD30,000 or less): {a}\
        \nUSD30,000 and USD50,000: {b}\
        \nUSD50,000 and USD100,000: {c}\
        \nUSD100,000 and USD200,000: {d}\
        \nUSD200,000 or more: {e}"\
        .format(a=x['income']['USD30,000 or less'],\
        b=x['income']['USD30,000 and USD50,000'], \
        c=x['income']['USD50,000 and USD100,000'],
        d=x['income']['USD100,000 and USD200,000'],
        e=x['income']['USD200,000 or more']))
    print("---------------------------------------------------------") 

Zip Code: 94022 
Population: 19,310 
Density 1,104.17 
The income(USD30,000 or less): 4.51%        
USD30,000 and USD50,000: 5.67%        
USD50,000 and USD100,000: 11.6%        
USD100,000 and USD200,000: 28.45%        
USD200,000 or more: 49.77%
---------------------------------------------------------
Zip Code: 94024 
Population: 22,536 
Density 3,086.45 
The income(USD30,000 or less): 2.58%        
USD30,000 and USD50,000: 4.99%        
USD50,000 and USD100,000: 13.28%        
USD100,000 and USD200,000: 31.08%        
USD200,000 or more: 48.07%
---------------------------------------------------------
Zip Code: 94040 
Population: 33,918 
Density 9,194.61 
The income(USD30,000 or less): 11.93%        
USD30,000 and USD50,000: 8.22%        
USD50,000 and USD100,000: 21.89%        
USD100,000 and USD200,000: 30.68%        
USD200,000 or more: 27.29%
---------------------------------------------------------
Zip Code: 94041 
Population: 12,929 
Density 8,451.43 
The income(USD30,000 or 

## ELT 3. 
## Step 1. Extract demographics data from mapszipcode.com to upload into a  Postgres database

In [42]:
zip_demo_dict=[]
 
for zipcode in zipcodes:
    tables = pd.read_html('http://www.mapszipcode.com/california/ladera%20ranch/'+ zipcode + '/demographics/')
    df2 = tables[1]
    df2.columns = ['Racial makeup', 'Population']
    df2.set_index("Racial makeup", inplace = True)
    dict1= (df2.to_dict())['Population']
    dict1['zipcode'] = zipcode
    zip_demo_dict.append(dict1)
zip_demo_dict

[{'White population': 12888,
  'Black or African American population': 27,
  'Asian population': 5467,
  'American Indian and Alaska Native population': 10,
  'Native Hawaiian and other Pacific Islander population': 0,
  'Other population': 148,
  'zipcode': '94022'},
 {'White population': 15696,
  'Black or African American population': 194,
  'Asian population': 5435,
  'American Indian and Alaska Native population': 27,
  'Native Hawaiian and other Pacific Islander population': 45,
  'Other population': 121,
  'zipcode': '94024'},
 {'White population': 19502,
  'Black or African American population': 790,
  'Asian population': 8530,
  'American Indian and Alaska Native population': 24,
  'Native Hawaiian and other Pacific Islander population': 187,
  'Other population': 3319,
  'zipcode': '94040'},
 {'White population': 8411,
  'Black or African American population': 461,
  'Asian population': 2861,
  'American Indian and Alaska Native population': 14,
  'Native Hawaiian and other P

## Step 2. Clean and structure the data

In [43]:
zipcode_data = pd.DataFrame(zip_demo_dict)
zipcode_data.dropna()

,White population,Black or African American population,Asian population,American Indian and Alaska Native population,Native Hawaiian and other Pacific Islander population,Other population,zipcode
0,12888.0,27.0,5467.0,10.0,0.0,148.0,94022
1,15696.0,194.0,5435.0,27.0,45.0,121.0,94024
2,19502.0,790.0,8530.0,24.0,187.0,3319.0,94040
3,8411.0,461.0,2861.0,14.0,0.0,696.0,94041
4,16416.0,677.0,7240.0,91.0,181.0,1362.0,94043
6,20123.0,1017.0,18651.0,70.0,204.0,3880.0,94086
7,25782.0,704.0,23311.0,109.0,142.0,1434.0,94087
8,8975.0,394.0,8597.0,112.0,12.0,1430.0,94089
9,12745.0,371.0,2921.0,15.0,0.0,136.0,94301
10,2480.0,147.0,1004.0,0.0,0.0,17.0,94304


## Step 3. Load the data into Postgres database

In [44]:
zipcode_data.to_sql(name='demographic_zipcode', con=engine, if_exists='replace', index=False)

In [45]:
pd.read_sql_query('select * from demographic_zipcode', con=engine).head()

,White population,Black or African American population,Asian population,American Indian and Alaska Native population,Native Hawaiian and other Pacific Islander population,Other population,zipcode
0,12888.0,27.0,5467.0,10.0,0.0,148.0,94022
1,15696.0,194.0,5435.0,27.0,45.0,121.0,94024
2,19502.0,790.0,8530.0,24.0,187.0,3319.0,94040
3,8411.0,461.0,2861.0,14.0,0.0,696.0,94041
4,16416.0,677.0,7240.0,91.0,181.0,1362.0,94043
